In [1]:
import pandas as pd
import regex as re
import glob
from mangoCR import pdf2image_ocr_text
## Importing necessary libraries

In [20]:
target_pdfs = glob.glob("bx_cb6_meeting_minutes/*.pdf")
target_pdfs
## Glob together the PDFs of the meeting minutes I previously scraped in my web scrape code.

['bx_cb6_meeting_minutes/may_22_2018_general_board_final.pdf',
 'bx_cb6_meeting_minutes/April-23-2024-Meeting-Minutes.pdf',
 'bx_cb6_meeting_minutes/may_28_2019_general_board_final.pdf',
 'bx_cb6_meeting_minutes/october_23_2018_general_board_draft.pdf',
 'bx_cb6_meeting_minutes/minutes-20200128.pdf',
 'bx_cb6_meeting_minutes/Board_Meeting_Minutes_4-26-22.pdf',
 'bx_cb6_meeting_minutes/march-26-2024-meeting-minutes.pdf',
 'bx_cb6_meeting_minutes/October-24-2023-Meeting-Minutes-Attendance.pdf',
 'bx_cb6_meeting_minutes/march_26_2019_general_board_final.pdf',
 'bx_cb6_meeting_minutes/4-27-21-cb4-full-board-meeting-minutes.pdf',
 'bx_cb6_meeting_minutes/november_27_2018_general_board_minutes.pdf',
 'bx_cb6_meeting_minutes/april_24_%202018_general_board_final.pdf',
 'bx_cb6_meeting_minutes/November-28-2023-General-Board-Meeting-Minutes.pdf',
 'bx_cb6_meeting_minutes/Board-Meeting%20Minutes-2-22-22.pdf',
 'bx_cb6_meeting_minutes/january_26_2021_cb4_full_board_minutes_v1.pdf',
 'bx_cb6_meetin

In [21]:
text = pdf2image_ocr_text(target_pdfs[-1])
text
## Using OCR on one PDF to check if its working correctly.

Processing PDF 1 of 1: January-23-2024-General-Board-Meeting-Minutes
  - Processed page 1 of 6 in January-23-2024-General-Board-Meeting-Minutes
  - Processed page 2 of 6 in January-23-2024-General-Board-Meeting-Minutes
  - Processed page 3 of 6 in January-23-2024-General-Board-Meeting-Minutes
  - Processed page 4 of 6 in January-23-2024-General-Board-Meeting-Minutes
  - Processed page 5 of 6 in January-23-2024-General-Board-Meeting-Minutes
  - Processed page 6 of 6 in January-23-2024-General-Board-Meeting-Minutes
Finished processing January-23-2024-General-Board-Meeting-Minutes

All PDFs have been processed.


'## January-23-2024-General-Board-Meeting-Minutes_page_1\nThe City of New York\nBronx Community Board Four\nPublic Hearing/General Board Meeting\nTuesday, January 23, 2024 | 6 pm\nBronx Museum of the Arts\n1040 Grand Concourse, Bronx, NY\nMR. ROBERT GARMENDIZ, BOARD CHAIR, PRESIDING\n\nMEMBERS PRESENT:\n\nMs. Ellen D. Abraham, Ms. Priscilla Alexander, Ms. Shekera Algarin, Ms. Rasheida Alston, Ms. Beverly\nBond, Mr. Dustin L. Carter, Ms. Triciah Claxton, Mrs. Rossmery Dais, Ms. Leticia N. Dinkins, Ms. Brenda\nDock, Mr. Robert Garmendiz, Ms. Veronique Green, Ms. Frances Hudson-Marrow, Ms. Towona Joye,\nMs. Nurys Laureano, Ms. Veronie Lawrence, Ms. Prettystar Lopez, Mr. Julian D. Lopez-Leyva, Mr.\nMohammed Mardah, Ms. Marie McCullough, Ms. Monique McMahon-Stancil, Mr. Leandro Monzon, Ms.\nMichelle Mootreddy, Mr. Ebrahim Dawda Ndure, Ms. Martha Reyes, Ms. Yohanny Rincon, Ms. Elba\nRodriguez, Ms. Lilliam Rosa, Rev. Julian Stephenson, Mr. Jackson Strong, Ms. Jessica Toro, Ms.\nKinanda Willia

In [4]:
from fuzzy_context_finder import keyword_context_finder
## Import fuzzy to do context finder

In [35]:
keywords = [
    " bridge",
    "Session:",
    "rehabilitation",
    "expressway",
    "highway",
           ]
## I chose these keywords to target mentions of bridges, repair projects and any comments made during the public session part of the meetings.

In [37]:
df = keyword_context_finder(text, keywords, 'bx_cb6_meeting_minutes/January-23-2024-General-Board-Meeting-Minutes.pdf', match_threshold=90  )
df
## Seeing how it will look in a df. Do I have the right columns, etc.

,File Name,Page Marker,Page Number,Matched Term,Original Term,Similarity Score,Search Term with 50 Words Context,Previous 250 Words (Including Term),Next 250 Words (Including Term)
0,bx_cb6_meeting_minutes/January-23-2024-General...,January-23-2024-General-Board-Meeting-Minutes_...,3,Session:,Session:,100.0,"Inc., Erwin Contreras-Community Resident, Fran...","Grand Concourse library branch, Commanding Dep...","Session: 1. Shakeema Rogers, The New York Publ..."


In [38]:
from tabular_2_text import tab2text
## Import tab2text to bring the df into a text file. Again doing all this on one PDF so I can hammer out any issues before I iterate through the entire glob of PDFs.

In [39]:
tab2text(df, "bxcb06_jan_24.txt")

bxcb06_jan_24.txt is in your local project folder.


'bxcb06_jan_24.txt'

In [41]:
## This worked!! Might not need 250 words after, but will keep for now. May change later

In [42]:
df_list = []
empty_list = []
for count, target_pdf in enumerate(target_pdfs, start=1):  # Use enumerate for progress tracking
    print(f"Processing file {count}/{len(target_pdfs)}: {target_pdf}")
    
    text = pdf2image_ocr_text(target_pdf)  # Extract text from the PDF
    
    try:
        # Append the DataFrame with results to df_list
        df_list.append(keyword_context_finder(text, keywords, target_pdf, match_threshold=85)), ## using default threshold of 85 to try to get other spellings and related words. Casting a semi-wide net to try to grab any instance of talk of bridges or infrastructure projects.
    except:
        # Handle files where no matches are found or other errors occur
        print(f"No words found in {target_pdf}")
        empty_list.append(target_pdf)
        
print("DONE PROCESSING!")

Processing file 1/44: bx_cb6_meeting_minutes/may_22_2018_general_board_final.pdf
Processing PDF 1 of 1: may_22_2018_general_board_final
  - Processed page 1 of 6 in may_22_2018_general_board_final
  - Processed page 2 of 6 in may_22_2018_general_board_final
  - Processed page 3 of 6 in may_22_2018_general_board_final
  - Processed page 4 of 6 in may_22_2018_general_board_final
  - Processed page 5 of 6 in may_22_2018_general_board_final
  - Processed page 6 of 6 in may_22_2018_general_board_final
Finished processing may_22_2018_general_board_final

All PDFs have been processed.
Processing file 2/44: bx_cb6_meeting_minutes/April-23-2024-Meeting-Minutes.pdf
Processing PDF 1 of 1: April-23-2024-Meeting-Minutes
  - Processed page 1 of 5 in April-23-2024-Meeting-Minutes
  - Processed page 2 of 5 in April-23-2024-Meeting-Minutes
  - Processed page 3 of 5 in April-23-2024-Meeting-Minutes
  - Processed page 4 of 5 in April-23-2024-Meeting-Minutes
  - Processed page 5 of 5 in April-23-2024-Meet

In [43]:
df = pd.concat(df_list, ignore_index = True)
df
## Concatenate all the dfs together into a final df. Then going to bring this into a txt file using tab2text.

,File Name,Page Marker,Page Number,Matched Term,Original Term,Similarity Score,Search Term with 50 Words Context,Previous 250 Words (Including Term),Next 250 Words (Including Term)
0,bx_cb6_meeting_minutes/may_22_2018_general_boa...,may_22_2018_general_board_final_page_3,3,Session,Session:,93.333333,"Public Session 1. Dr. James Fairbanks, Residen...",Public Session,"Session 1. Dr. James Fairbanks, Resident — The..."
1,bx_cb6_meeting_minutes/April-23-2024-Meeting-M...,April-23-2024-Meeting-Minutes_page_3,3,Session:,Session:,100.000000,"Esther Polanco-Volunteers of America, Michael ...","Shala-NYPD 44 Precinct, Abbygail Brown, Susann...","Session: 1.) lolanie de Los Santos, 24 and 24 ..."
2,bx_cb6_meeting_minutes/April-23-2024-Meeting-M...,April-23-2024-Meeting-Minutes_page_3,3,session,Session:,93.333333,"his wife/board member, Rossmery Dais, talked a...","Shala-NYPD 44 Precinct, Abbygail Brown, Susann...","session with Sedgwick Houses, introduced his e..."
3,bx_cb6_meeting_minutes/April-23-2024-Meeting-M...,April-23-2024-Meeting-Minutes_page_4,4,sessions,Session:,87.500000,"On a different note, over the last several mon...","Led and delivered by Thomas Alexander, Distric...",sessions for the board. He noted we need parli...
4,bx_cb6_meeting_minutes/may_28_2019_general_boa...,may_28_2019_general_board_final_page_3,3,Session,Session:,93.333333,Public Session - None Welcoming remarks and an...,Public Session,Session - None Welcoming remarks and announcem...
...,...,...,...,...,...,...,...,...,...
140,bx_cb6_meeting_minutes/september_24_2019_gener...,september_24_2019_general_board_draft_page_4,4,Bridge,bridge,92.307692,Parks Department. The change of tennis fees fr...,to replace the windows in the common areas to ...,Bridge Park which is tabled at this time. Ther...
141,bx_cb6_meeting_minutes/september_24_2019_gener...,september_24_2019_general_board_draft_page_8,8,Bridge,bridge,92.307692,e Parks — Scoping Meetings o Bridge Park o Ogd...,e Parks — Scoping Meetings o Bridge,Bridge Park o Ogden/Plimpton playground openin...
142,bx_cb6_meeting_minutes/september_24_2019_gener...,september_24_2019_general_board_draft_page_8,8,Highway,highway,100.000000,e Parks — Scoping Meetings o Bridge Park o Ogd...,e Parks — Scoping Meetings o Bridge Park o Ogd...,"Highway = 458 students o 97 W. 167"" Street 100..."
143,bx_cb6_meeting_minutes/september_24_2019_gener...,september_24_2019_general_board_draft_page_8,8,Bridge,bridge,92.307692,Understand your rights and responsibilities o ...,e Parks — Scoping Meetings o Bridge Park o Ogd...,"Bridge Builders located at 156 W. 164"" Street ..."


In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145 entries, 0 to 144
Data columns (total 9 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   File Name                            145 non-null    object 
 1   Page Marker                          145 non-null    object 
 2   Page Number                          145 non-null    int64  
 3   Matched Term                         145 non-null    object 
 4   Original Term                        145 non-null    object 
 5   Similarity Score                     145 non-null    float64
 6   Search Term with 50 Words Context    145 non-null    object 
 7   Previous 250 Words (Including Term)  145 non-null    object 
 8   Next 250 Words (Including Term)      145 non-null    object 
dtypes: float64(1), int64(1), object(7)
memory usage: 10.3+ KB


In [45]:
tab2text(df, "v1_bxcb06_cbmeetings_bridge+terms.txt")

v1_bxcb06_cbmeetings_bridge+terms.txt is in your local project folder.


'v1_bxcb06_cbmeetings_bridge+terms.txt'

In [46]:
## this can act as version 1 -- with room to improve via different keywords and perhaps different match threshold. On a future project, I might use a threshold of 90+ to narrow the results a bit. Most findings with a threshold of <90 didn't feel that useful to me.